In [1]:
# Idee: Peak dB Analysis

import os
import numpy as np
import pandas as pd
from scipy.io import wavfile

# Lade die df_dataset-Datei
df_dataset_path = r"df_dataset.xlsx"  # Passe den Pfad an
df_dataset = pd.read_excel(df_dataset_path)

# Pfad zu den vorverarbeiteten Audiodateien
audio_directory = r'/Users/jakob/Downloads/IDMT_Traffic/preprocessed_audio_not_normalized'

# Funktion zur Berechnung des RMS-Werts
def calculate_rms(file_path):
    """
    Berechnet den RMS-Wert einer Audiodatei.
    Args:
        file_path (str): Pfad zur Audiodatei.
    Returns:
        float: RMS-Wert.
    """
    # Lade die Audiodatei
    sample_rate, data = wavfile.read(file_path)
    if data.ndim == 2:  # Falls Stereo, konvertiere zu Mono
        data = np.mean(data, axis=1)
    # Berechne den RMS-Wert
    rms = np.sqrt(np.mean(data**2))
    return rms

# Liste für RMS-Werte
rms_data = []

# Berechne RMS für jede Datei im Verzeichnis
for file_name in os.listdir(audio_directory):
    if file_name.endswith(".wav"):
        # Berechne RMS
        file_path = os.path.join(audio_directory, file_name)
        rms_value = calculate_rms(file_path)
        
        # Entferne ".wav" aus dem Dateinamen für das Merging
        base_name = file_name.replace(".wav", "")
        
        # Speichere den RMS-Wert und den Dateinamen
        rms_data.append({'file': base_name, 'rms': rms_value})

# Konvertiere RMS-Daten in DataFrame
df_rms = pd.DataFrame(rms_data)

# Merge mit df_dataset anhand der Spalte "file"
df_merged = pd.merge(df_dataset, df_rms, on='file', how='left')

# Speichere die erweiterte Tabelle
output_path = r"df_dataset_with_rms.xlsx"  # Passe den Pfad an
df_merged.to_excel(output_path, index=False)
print(f"Erweiterte Tabelle mit RMS-Werten gespeichert unter: {output_path}")

Erweiterte Tabelle mit RMS-Werten gespeichert unter: df_dataset_with_rms.xlsx


In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load the dataset
df = pd.read_excel(r"df_dataset_with_rms.xlsx")

# Create a subplot with 1 row and 3 columns
fig = make_subplots(
    rows=1, cols=3, 
    subplot_titles=("30 km/h", "50 km/h", "70 km/h"), 
    horizontal_spacing=0.1
)

# Speeds to iterate
speeds = [30, 50, 70]

# Loop through speeds and add box plots to subplots
for i, speed in enumerate(speeds):
    # Filter the data for the current speed
    df_filtered = df[(df['speed_kmh'] == str(speed)) & (df['microphone'] == "SE") & (df['vehicle'] != "M")]
    
    # Create a box plot for the current speed
    fig.add_trace(
        go.Box(
            x=df_filtered['vehicle'],
            y=df_filtered['rms'],
            name=f"{speed} km/h",  # Legend label
            marker=dict(color='blue'),  # Optional: Color
            boxmean=True  # Show mean in the box plot
        ),
        row=1, col=i + 1  # Place in the corresponding subplot
    )

# Update layout for spacing and size
fig.update_layout(
    title_text="RMS in Different Vehicle Categories Across Speeds",
    height=500,
    width=1200,  # Adjust width for side-by-side view
    showlegend=False  # Hide redundant legend
)

# Show the figure
fig.show()

In [7]:
# Die Variable „df“ wurde aus dem URI geladen: /Users/jakob/Library/CloudStorage/OneDrive-student.kit.edu/Studium/02_Master/4. Semester/seminar/RTN-jakob/df_dataset_merged.xlsx
import pandas as pd
# Load the dataset
df = pd.read_excel(r"df_dataset_with_rms.xlsx")

# Zeilen basierend auf Spalte filtern: 'vehicle'
df = df[df['vehicle'] == "C"]

# Zeilen basierend auf Spalte filtern: 'microphone'
df = df[df['microphone'] == "SE"]

import pandas as pd
import plotly.express as px

# Erstelle den Boxplot
fig = px.box(
    df,
    x='speed_kmh',  # Fahrzeugklassen (z. B. Bus, Car, Motorcycle, Truck)
    y='rms',
    title='RMS for Car at 30, 50, 70kmh',
    labels={'vehicle': 'Vehicle Class', 'rms': 'Root Mean Square'},
    color='vehicle',  # Optional: Farben für verschiedene Fahrzeugklassen
    category_orders={'speed_kmh': [30, 50, 70]}  # Feste Reihenfolge für die x-Achse
)

fig.update_layout(
    height=500,
    width=500,  # Adjust width for side-by-side view
    showlegend=False  # Hide redundant legend
)

# Zeige den Plot
fig.show()

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.io import wavfile

def calculate_peak_loudness(file_path):
    """
    Calculate peak loudness (dBFS) for an audio file.
    Args:
        file_path (str): Path to the WAV audio file.
    Returns:
        float: Peak loudness in dBFS.
    """
    # Load audio
    sample_rate, data = wavfile.read(file_path)

    # Convert to mono if stereo
    if len(data.shape) == 2:
        data = np.mean(data, axis=1).astype(data.dtype)

    # Calculate peak amplitude
    peak_amplitude = np.max(np.abs(data))

    # Define full-scale amplitude based on audio format
    if np.issubdtype(data.dtype, np.integer):
        full_scale_amplitude = np.iinfo(data.dtype).max
    elif np.issubdtype(data.dtype, np.floating):
        full_scale_amplitude = 1.0  # Assuming normalized float format
    else:
        raise ValueError(f"Unsupported data type: {data.dtype}")

    # Calculate peak loudness in dBFS
    if peak_amplitude == 0:
        peak_loudness = -np.inf  # Handle silence case
    else:
        peak_loudness = 20 * np.log10(peak_amplitude / full_scale_amplitude)

    return peak_loudness

# Path to the dataset and original audio files
df_dataset_path = "df_dataset.xlsx"  # Path to the dataset
audio_directory = "/Users/jakob/Downloads/IDMT_Traffic/audio"  # Original audio files directory

# Load the dataset
df_dataset = pd.read_excel(df_dataset_path)

# Initialize a list to store peak loudness data
peak_loudness_data = []

# Loop through each audio file in the directory
for file_name in os.listdir(audio_directory):
    if file_name.endswith(".wav"):
        file_path = os.path.join(audio_directory, file_name)
        
        # Calculate peak loudness
        try:
            peak_loudness = calculate_peak_loudness(file_path)
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
            continue

        # Remove ".wav" for merging compatibility
        base_name = file_name.replace(".wav", "")
        
        # Append results to the list
        peak_loudness_data.append({'file': base_name, 'peak_loudness': peak_loudness})

# Convert the peak loudness data to a DataFrame
df_peak_loudness = pd.DataFrame(peak_loudness_data)

# Merge with the original dataset based on the 'file' column
df_dataset_peak_loudness = pd.merge(df_dataset, df_peak_loudness, on='file', how='left')

# Save the merged dataset to a new Excel file
output_path = "df_dataset_peak_loudness.xlsx"
df_dataset_peak_loudness.to_excel(output_path, index=False)

print(f"Merged dataset with peak loudness saved to: {output_path}")


Merged dataset with peak loudness saved to: df_dataset_peak_loudness.xlsx


In [7]:
# Die Variable „df“ wurde aus dem URI geladen: /Users/jakob/Library/CloudStorage/OneDrive-student.kit.edu/Studium/02_Master/4. Semester/seminar/RTN-jakob/df_dataset_merged.xlsx
import pandas as pd
# Load the dataset
df = pd.read_excel(r"df_dataset_peak_loudness.xlsx")

# Zeilen basierend auf Spalte filtern: 'vehicle'
df = df[df['vehicle'] == "T"]

# Zeilen basierend auf Spalte filtern: 'microphone'
df = df[df['microphone'] == "SE"]

import pandas as pd
import plotly.express as px

# Erstelle den Boxplot
fig = px.box(
    df,
    x='speed_kmh',  # Fahrzeugklassen (z. B. Bus, Car, Motorcycle, Truck)
    y='peak_loudness',
    title='Peak Loudness for Car at 30, 50, 70kmh',
    category_orders={'speed_kmh': [30, 50, 70]}  # Feste Reihenfolge für die x-Achse
)

fig.update_layout(
    height=500,
    width=500,  # Adjust width for side-by-side view
    showlegend=False  # Hide redundant legend
)

# Zeige den Plot
fig.show()